In [258]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#take ratings from csv file 
ratings = pd.read_csv('ratings.csv')



In [259]:
def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    # Frequencies of each value in the column.
    freq = df[column].value_counts()
    # Select frequent values. Value is in the index.
    frequent_values = freq[freq >= min_freq].index
    # Return only rows with value frequency above threshold.
    return df[df[column].isin(frequent_values)]


In [260]:
def train_and_test_split(ratings,trainSize):
    train, test = train_test_split(ratings, test_size=0.1,train_size=trainSize,random_state=1)
    matrixTest = test.pivot(index='userId', columns='movieId', values='rating')
    matrixTrain = train.pivot(index='userId', columns='movieId', values='rating')
    return matrixTest,matrixTrain
        

In [261]:
def setZeros(matrixTest,matrixTrain):
    matrixTrain.fillna(0,inplace=True)
    matrixTest.fillna(0,inplace=True)
    

In [262]:
def pearsonSimilarity(matrixTrain):
    return matrixTrain.corr()

In [263]:
def topIndexSimilarities(movie,k,itemSimilarity):
    indexes = itemSimilarity[movie].sort_values(ascending=False)[:k].index
    return indexes
    


In [264]:
def topSimilarities_Weights(movieId,k,itemSimilarity):
    similarities = itemSimilarity[movieId].sort_values(ascending=False)[:k].values
    return similarities

In [265]:
def averagePrediction(userId,movieId,matrixTrain,matrixTest,indexes,ABS_Err_Average,total_counter,TP,FP,TN,FN):
    counter = 0
    sum = 0
    averageRating = 0
    
    for similarMovie in indexes:
        if(matrixTrain[similarMovie][userId]!=0):
            sum += matrixTrain[similarMovie][userId]
            counter += 1
            
            averageRating = sum/counter
            
    if(averageRating!=0): #PRINT MONO OSES MPOREI NA PROBLEPSEI
        print("AVERAGE:  User:",userId,"Movie:  ",movieId," Rate: ",averageRating)
        
      
        
    else:
        print("Cannot calculate Movie: ",movieId,"from User: ",userId,"with AVERAGE")
        
    #calculating MAE 
        
    summary_ABS_Error = calcABS_Error(averageRating,userId,movieId,matrixTest,ABS_Err_Average)
    MAE = calcMAE(summary_ABS_Error,total_counter)
    
    
    
    
    #classification of trues and falses
    truePositive,trueNegative,falseNegative,falsePositive = classification(matrixTest,userId,movieId,averageRating,TP,FP,TN,FN)
    
    
    #calculating precision
    precision = calcPrecision(truePositive,falsePositive)
    recall = calcRecall(truePositive,falseNegative)
    
    
    return MAE,precision,recall
    
            
    
    
    

In [266]:
def weightedAveragePrediction(userId,movieId,matrixTrain,matrixTest,indexes,similarities,k,ABS_Err_Weighted_Average,total_counter,MAE,TP,FP,TN,FN):
    summary = 0
    weightedAverage = 0
    predictedRating = 0
    
    for i in range(k):
        if(matrixTrain[indexes[i]][userId]!=0):
            predictedRating += matrixTrain[indexes[i]][userId]*similarities[i]
            summary += similarities[i]
    
    if(summary!=0):
        weightedAverage = predictedRating/summary
        
    if(weightedAverage>=0 and weightedAverage<=5):
            print("WEIGHTED AVERAGE:  User:",userId,"Movie:  ",movieId," Rate: ",weightedAverage)
            
            
            
            
            
    else:
        print("Cannot calculate Movie: ",movieId,"from User: ",userId,"with WEIGHTED_AVERAGE")
        
    #calculating MAE
        
    summary_ABS_Error = calcABS_Error(weightedAverage,userId,movieId,matrixTest,ABS_Err_Weighted_Average)
    MAE = calcMAE(summary_ABS_Error,total_counter)
    
    
    
    
    
    #classification of trues and falses
    truePositive,trueNegative,falseNegative,falsePositive = classification(matrixTest,userId,movieId,weightedAverage,TP,FP,TN,FN)
    
    
    #calculating precision and recall
    precision = calcPrecision(truePositive,falsePositive)
    recall = calcRecall(truePositive,falseNegative)
    
    
    return MAE,precision,recall
    
   
    
    

In [267]:
def calcABS_Error(rating,userId,movieId,matrixTest,ABS_Err):
    absErr = abs(rating-matrixTest[movieId][userId])
    #store it in array,in order to calculate the summary easier without losing it
    ABS_Err.append(absErr)
    return ABS_Err
    
    

In [268]:
def calcMAE(sumABS_Error,counter):
    Sum = sum(sumABS_Error)
    if(counter!=0):
        return (Sum/counter)

In [269]:
def isRelevant(rating):
    if(rating>=3):
        return True
    else:
        return False

In [270]:
def classification(matrixTest,userId,movieId,rating,truePositive,falsePositive,trueNegative,falseNegative):
    
    
    if(isRelevant(rating) and isRelevant(matrixTest[movieId][userId])):
        truePositive.append(1)
    elif(not isRelevant(rating) and not isRelevant(matrixTest[movieId][userId])):
        trueNegative.append(1)
    elif(not isRelevant(rating) and isRelevant(matrixTest[movieId][userId])):
        falseNegative.append(1)
    elif(isRelevant(rating) and not isRelevant(matrixTest[movieId][userId])):
        falsePositive.append(1)
        
        
    #find total sum of arrays 
    totalTruePositives = sum(truePositive)
    totalTrueNegatives = sum(trueNegative)
    totalFalseNegatives = sum(falseNegative)
    totalFalsePositives = sum(falsePositive)
        
    return totalTruePositives,totalTrueNegatives,totalFalseNegatives,totalFalsePositives
        
        
    

In [271]:
def calcPrecision(TP,FP):
    if((TP+FP)!=0):
        return TP/(TP+FP)
        
    

In [272]:
def calcRecall(TP,FN):
    if((TP+FN)!=0):
        return TP/(TP+FN)

In [278]:
def main(ratings,k,trainSize):
    #filtering
    ratings = filter_by_freq(ratings,'movieId',5)
    ratings = filter_by_freq(ratings,'userId',5)
    
    #splitting 
    matrixTest,matrixTrain = train_and_test_split(ratings,trainSize)
    
    #setting Nan = 0 
    setZeros(matrixTest,matrixTrain)
    
    #finding pearson similarity
    itemSimilarity = pearsonSimilarity(matrixTrain)
    
    #declare an array to calculate later the MAE
    ABS_Err_Average = []
    ABS_Err_Weighted_Average = []
    
    #total_counter of movies
    total_counter = 0
    
    MAE = None
    
    #setting arrays in order to calculate later precision,and recall
    TP_A=[]
    FP_A=[]
    TN_A=[]
    FN_A=[]
    
    TP_WA=[]
    FP_WA=[]
    TN_WA=[]
    FN_WA=[]
    
    
   
    
    #the program starts
    
    for user in matrixTest.index:
        for movie in matrixTest.columns:
            if(matrixTest[movie][user]!=0):
                userId = user
                movieId = movie
                
                indexes = topIndexSimilarities(movieId,k,itemSimilarity)
                similarities = topSimilarities_Weights(movieId,k,itemSimilarity)
                
                total_counter +=1
                
                print("-------------------------------------------")
                
                #calculate Average Prediction
                MAE_Average,precision_Average,recall_Average = averagePrediction(userId,movieId,matrixTrain,matrixTest,indexes,ABS_Err_Average,total_counter,TP_A,FP_A,TN_A,FN_A)
              
                
                #calculate Weighted Average Prediction
                MAE_WeightedAverage,precision_WeightedAverage,recall_WeightedAverage = weightedAveragePrediction(userId,movieId,matrixTrain,matrixTest,indexes,similarities,k,ABS_Err_Weighted_Average,total_counter,MAE,TP_WA,FP_WA,TN_WA,FN_WA)
                
                
    print("-------------------------------------------")            
    print("MAE OF AVERAGE:",MAE_Average)
    print("MAE OF WEIGHTED AVERAGE:",MAE_WeightedAverage)
    
    print("-------------------------------------------")  
    
    print("PRECISION OF AVERAGE: ",precision_Average)
    print("PRECISION OF WEIGHTED AVERAGE: ",precision_WeightedAverage)
    
    print("-------------------------------------------")  
    

    print("RECALL OF AVERAGE: ",recall_Average)
    print("RECALL OF AVERAGE: ",recall_WeightedAverage)
    
    
    
    
    
    
    

In [279]:
#execute (file,k,trainSize(ex:0.9 for 90% training split))
main(ratings,50,0.9)

-------------------------------------------
AVERAGE:  User: 1 Movie:   3  Rate:  4.2
WEIGHTED AVERAGE:  User: 1 Movie:   3  Rate:  4.216593147962053
-------------------------------------------
AVERAGE:  User: 1 Movie:   50  Rate:  4.5
WEIGHTED AVERAGE:  User: 1 Movie:   50  Rate:  4.506313080769139
-------------------------------------------
AVERAGE:  User: 1 Movie:   163  Rate:  3.875
WEIGHTED AVERAGE:  User: 1 Movie:   163  Rate:  3.859922304823002
-------------------------------------------
AVERAGE:  User: 1 Movie:   356  Rate:  4.3125
WEIGHTED AVERAGE:  User: 1 Movie:   356  Rate:  4.242548882549621
-------------------------------------------
AVERAGE:  User: 1 Movie:   441  Rate:  4.428571428571429
WEIGHTED AVERAGE:  User: 1 Movie:   441  Rate:  4.453847263764656
-------------------------------------------
AVERAGE:  User: 1 Movie:   552  Rate:  4.0
WEIGHTED AVERAGE:  User: 1 Movie:   552  Rate:  4.024035035167323
-------------------------------------------
AVERAGE:  User: 1 Movie: 

-------------------------------------------
AVERAGE:  User: 6 Movie:   416  Rate:  3.2857142857142856
WEIGHTED AVERAGE:  User: 6 Movie:   416  Rate:  3.2940574274076146
-------------------------------------------
AVERAGE:  User: 6 Movie:   434  Rate:  3.888888888888889
WEIGHTED AVERAGE:  User: 6 Movie:   434  Rate:  3.8578530788479806
-------------------------------------------
AVERAGE:  User: 6 Movie:   455  Rate:  4.111111111111111
WEIGHTED AVERAGE:  User: 6 Movie:   455  Rate:  4.10436173563697
-------------------------------------------
AVERAGE:  User: 6 Movie:   457  Rate:  3.888888888888889
WEIGHTED AVERAGE:  User: 6 Movie:   457  Rate:  3.8789751133249357
-------------------------------------------
AVERAGE:  User: 6 Movie:   509  Rate:  3.6363636363636362
WEIGHTED AVERAGE:  User: 6 Movie:   509  Rate:  3.615398012960965
-------------------------------------------
AVERAGE:  User: 6 Movie:   548  Rate:  3.0
WEIGHTED AVERAGE:  User: 6 Movie:   548  Rate:  3.0
----------------------

-------------------------------------------
AVERAGE:  User: 14 Movie:   153  Rate:  3.1333333333333333
WEIGHTED AVERAGE:  User: 14 Movie:   153  Rate:  3.121021677300341
-------------------------------------------
AVERAGE:  User: 14 Movie:   288  Rate:  3.625
WEIGHTED AVERAGE:  User: 14 Movie:   288  Rate:  3.6715476619704277
-------------------------------------------
Cannot calculate Movie:  351 from User:  14 with AVERAGE
WEIGHTED AVERAGE:  User: 14 Movie:   351  Rate:  0
-------------------------------------------
AVERAGE:  User: 14 Movie:   367  Rate:  3.1538461538461537
WEIGHTED AVERAGE:  User: 14 Movie:   367  Rate:  3.0832107226232397
-------------------------------------------
AVERAGE:  User: 14 Movie:   540  Rate:  3.6
WEIGHTED AVERAGE:  User: 14 Movie:   540  Rate:  3.6368302609754286
-------------------------------------------
AVERAGE:  User: 14 Movie:   593  Rate:  3.3333333333333335
WEIGHTED AVERAGE:  User: 14 Movie:   593  Rate:  3.374503511549945
-----------------------

-------------------------------------------
AVERAGE:  User: 19 Movie:   64  Rate:  1.9090909090909092
WEIGHTED AVERAGE:  User: 19 Movie:   64  Rate:  1.8326077965886685
-------------------------------------------
AVERAGE:  User: 19 Movie:   102  Rate:  2.2222222222222223
WEIGHTED AVERAGE:  User: 19 Movie:   102  Rate:  2.1150329327962596
-------------------------------------------
AVERAGE:  User: 19 Movie:   165  Rate:  2.6451612903225805
WEIGHTED AVERAGE:  User: 19 Movie:   165  Rate:  2.651578143521803
-------------------------------------------
AVERAGE:  User: 19 Movie:   173  Rate:  2.4285714285714284
WEIGHTED AVERAGE:  User: 19 Movie:   173  Rate:  2.480804498521392
-------------------------------------------
AVERAGE:  User: 19 Movie:   289  Rate:  2.375
WEIGHTED AVERAGE:  User: 19 Movie:   289  Rate:  2.371593570946208
-------------------------------------------
AVERAGE:  User: 19 Movie:   350  Rate:  2.388888888888889
WEIGHTED AVERAGE:  User: 19 Movie:   350  Rate:  2.3996742094

-------------------------------------------
AVERAGE:  User: 19 Movie:   3745  Rate:  3.6
WEIGHTED AVERAGE:  User: 19 Movie:   3745  Rate:  3.577583948094776
-------------------------------------------
AVERAGE:  User: 19 Movie:   3763  Rate:  2.5625
WEIGHTED AVERAGE:  User: 19 Movie:   3763  Rate:  2.5803225538329273
-------------------------------------------
AVERAGE:  User: 19 Movie:   3784  Rate:  2.0
WEIGHTED AVERAGE:  User: 19 Movie:   3784  Rate:  2.0
-------------------------------------------
AVERAGE:  User: 20 Movie:   2  Rate:  4.111111111111111
WEIGHTED AVERAGE:  User: 20 Movie:   2  Rate:  4.118269555458906
-------------------------------------------
AVERAGE:  User: 20 Movie:   34  Rate:  4.083333333333333
WEIGHTED AVERAGE:  User: 20 Movie:   34  Rate:  4.117109391601141
-------------------------------------------
AVERAGE:  User: 20 Movie:   594  Rate:  4.136363636363637
WEIGHTED AVERAGE:  User: 20 Movie:   594  Rate:  4.203453044683511
--------------------------------------

...

-------------------------------------------
AVERAGE:  User: 601 Movie:   1  Rate:  4.375
WEIGHTED AVERAGE:  User: 601 Movie:   1  Rate:  4.323903409100161
-------------------------------------------
AVERAGE:  User: 601 Movie:   318  Rate:  4.65
WEIGHTED AVERAGE:  User: 601 Movie:   318  Rate:  4.690755841510412
-------------------------------------------
AVERAGE:  User: 601 Movie:   1198  Rate:  4.75
WEIGHTED AVERAGE:  User: 601 Movie:   1198  Rate:  4.747185449544926
-------------------------------------------
AVERAGE:  User: 601 Movie:   1262  Rate:  4.25
WEIGHTED AVERAGE:  User: 601 Movie:   1262  Rate:  4.255424157757374
-------------------------------------------
AVERAGE:  User: 601 Movie:   2019  Rate:  4.25
WEIGHTED AVERAGE:  User: 601 Movie:   2019  Rate:  4.243957586085046
-------------------------------------------
AVERAGE:  User: 601 Movie:   4963  Rate:  4.555555555555555
WEIGHTED AVERAGE:  User: 601 Movie:   4963  Rate:  4.556199666047754
----------------------------------

-------------------------------------------
AVERAGE:  User: 603 Movie:   2731  Rate:  4.4
WEIGHTED AVERAGE:  User: 603 Movie:   2731  Rate:  4.4048861126676675
-------------------------------------------
AVERAGE:  User: 603 Movie:   2802  Rate:  3.1538461538461537
WEIGHTED AVERAGE:  User: 603 Movie:   2802  Rate:  3.168552032132647
-------------------------------------------
AVERAGE:  User: 603 Movie:   2875  Rate:  3.4615384615384617
WEIGHTED AVERAGE:  User: 603 Movie:   2875  Rate:  3.4809230836422693
-------------------------------------------
AVERAGE:  User: 603 Movie:   2883  Rate:  3.5
WEIGHTED AVERAGE:  User: 603 Movie:   2883  Rate:  3.3828601006409778
-------------------------------------------
AVERAGE:  User: 603 Movie:   2908  Rate:  3.4375
WEIGHTED AVERAGE:  User: 603 Movie:   2908  Rate:  3.4591769712095575
-------------------------------------------
AVERAGE:  User: 603 Movie:   2912  Rate:  4.294117647058823
WEIGHTED AVERAGE:  User: 603 Movie:   2912  Rate:  4.28214866626

-------------------------------------------
AVERAGE:  User: 606 Movie:   2997  Rate:  3.9310344827586206
WEIGHTED AVERAGE:  User: 606 Movie:   2997  Rate:  3.9418309370772495
-------------------------------------------
AVERAGE:  User: 606 Movie:   3033  Rate:  3.625
WEIGHTED AVERAGE:  User: 606 Movie:   3033  Rate:  3.6233783959160277
-------------------------------------------
AVERAGE:  User: 606 Movie:   3081  Rate:  3.5476190476190474
WEIGHTED AVERAGE:  User: 606 Movie:   3081  Rate:  3.5755368568234682
-------------------------------------------
AVERAGE:  User: 606 Movie:   3083  Rate:  4.131578947368421
WEIGHTED AVERAGE:  User: 606 Movie:   3083  Rate:  4.127686937287701
-------------------------------------------
AVERAGE:  User: 606 Movie:   3105  Rate:  3.5
WEIGHTED AVERAGE:  User: 606 Movie:   3105  Rate:  3.511594195383475
-------------------------------------------
AVERAGE:  User: 606 Movie:   3168  Rate:  3.642857142857143
WEIGHTED AVERAGE:  User: 606 Movie:   3168  Rate:  3

-------------------------------------------
AVERAGE:  User: 608 Movie:   2094  Rate:  2.75
WEIGHTED AVERAGE:  User: 608 Movie:   2094  Rate:  2.762247835301125
-------------------------------------------
AVERAGE:  User: 608 Movie:   2193  Rate:  2.5
WEIGHTED AVERAGE:  User: 608 Movie:   2193  Rate:  2.484633855929396
-------------------------------------------
AVERAGE:  User: 608 Movie:   2454  Rate:  2.8333333333333335
WEIGHTED AVERAGE:  User: 608 Movie:   2454  Rate:  2.8296840400050933
-------------------------------------------
AVERAGE:  User: 608 Movie:   2560  Rate:  3.3461538461538463
WEIGHTED AVERAGE:  User: 608 Movie:   2560  Rate:  3.3120220830973675
-------------------------------------------
AVERAGE:  User: 608 Movie:   2640  Rate:  3.1739130434782608
WEIGHTED AVERAGE:  User: 608 Movie:   2640  Rate:  3.1357488207851496
-------------------------------------------
AVERAGE:  User: 608 Movie:   2707  Rate:  3.25
WEIGHTED AVERAGE:  User: 608 Movie:   2707  Rate:  3.248533731181

-------------------------------------------
AVERAGE:  User: 610 Movie:   5902  Rate:  3.9285714285714284
WEIGHTED AVERAGE:  User: 610 Movie:   5902  Rate:  3.937067423313652
-------------------------------------------
AVERAGE:  User: 610 Movie:   6157  Rate:  2.769230769230769
WEIGHTED AVERAGE:  User: 610 Movie:   6157  Rate:  2.7724533941318676
-------------------------------------------
AVERAGE:  User: 610 Movie:   6250  Rate:  2.9615384615384617
WEIGHTED AVERAGE:  User: 610 Movie:   6250  Rate:  3.026002333168059
-------------------------------------------
AVERAGE:  User: 610 Movie:   6283  Rate:  3.875
WEIGHTED AVERAGE:  User: 610 Movie:   6283  Rate:  3.883358725657598
-------------------------------------------
AVERAGE:  User: 610 Movie:   6502  Rate:  4.118421052631579
WEIGHTED AVERAGE:  User: 610 Movie:   6502  Rate:  4.14557066051079
-------------------------------------------
AVERAGE:  User: 610 Movie:   6595  Rate:  2.9411764705882355
WEIGHTED AVERAGE:  User: 610 Movie:   65